In [84]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from scipy import stats

# Linear Regression

$$f(X) = \beta_0 + \sum_{j=1}^p X_j \beta_j$$

### Residual sum of squares

$$\text{RSS}(\beta) = \sum_{i=1}^N  (y_i - f(X_i))^2$$

### Find $\beta$ to minimize RSS

$$X^T(y - X\beta) = 0$$
$$\hat{\beta} = (X^TX)^{-1}X^Ty$$

In [95]:
X = np.random.randn(313, 3) * 2.56 + 3.6
y = 1.34 * X[:, 0] + 2.67 * X[:, 1] + 7.89 + 0.3 * np.random.randn(len(X))
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.2)

def add_col1(m):
    return np.append(np.ones((len(m),1)), m, axis=1)

X_train2 = add_col1(X_train)
beta = np.linalg.inv(X_train2.T @ X_train2) @ X_train2.T @ y_train

print('intercept:', beta[0])
print('coeffs', beta[1:])
print('train error:', np.mean((y_train - X_train2 @ beta)**2))
print('test error:', np.mean((y_test - add_col1(X_test) @ beta)**2))

intercept: 7.852440105669972
coeffs [1.34469402 2.66954109 0.00335914]
train error: 0.09013343424315516
test error: 0.08305351785720833


Let's define $\sigma^2$ the constant variance of the obervations

$$\text{Var}(\hat{\beta}) = (X^TX)^{-1}\sigma^2$$
$$\hat{\sigma}^2 = \frac{1}{N - p - 1} \sum_{i=1}^N (y_i - \hat{y_i})^2$$

$$Y = \mathbb{E}(Y|X1,\text{...},X_p) + \epsilon, \space \epsilon \sim \mathcal{N}(0, \sigma^2)$$
$$\hat{\beta} \sim \mathcal{N}(\beta, (X^TX)^{-1}\sigma^2)$$

### Null hypotesis $\beta_j = 0$

$$z_j = \frac{\hat{\beta}}{\hat{\sigma} \sqrt{v_j}}, \space v_j = (X^TX)^{-1}_{jj}$$

If $|z_j|$ greater than a threshold, the null-hypothesis is rejected.

If $P(z > |z_j|)$ is greater than a threshold, the null-hypothesis is rejected.

$$P(z > |z_j|) = 2 * (1 - P(z < |z_j|))$$

$P(z < |z_j|)$ is the CDF of the t-distribution (can be computed with t-table or software packages)

### Significance test for a group of coefficient

F-statistic:
$$F = \frac{(\text{RSS}_0 - \text{RSS}_1)/(p_1-p_0)}{\text{RSS}_1/(N - p_1 - 1)}$$

- $\text{RSS}_1$: residual sum of square of complete model with $p_1 + 1$ parameters
- $\text{RSS}_0$: residual sum of square of reduced model with $p_0 + 1$ parameters (without params to be tested for significance)

## Standard error of parameters

$$\text{se}(\hat{b_j}) = \hat{\sigma} \sqrt{v_j}$$

$1 - 2\alpha$ confidence interval:
$$(\hat{\beta_j} - z^{1-\alpha} \text{se}(\hat{b_j}), \hat{\beta_j} + z^{1-\alpha} \text{se}(\hat{b_j}))$$

$z^{1-\alpha}$: $1-\alpha$ percentile of the normal distribution.

$$z^{1 - 0.025} = 1.96$$
$$z^{1 - 0.05} = 1.645$$

### Compute T-Score and P-Values

In [2]:
ds = load_diabetes()
X = ds.data
y = ds.target

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est = est.fit()
print(est.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     46.27
Date:                Fri, 28 Dec 2018   Prob (F-statistic):           3.83e-62
Time:                        14:52:44   Log-Likelihood:                -2386.0
No. Observations:                 442   AIC:                             4794.
Df Residuals:                     431   BIC:                             4839.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        152.1335      2.576     59.061      0.0

In [3]:
# Compute coeff

X_int = np.append(np.ones((len(X),1)), X, axis=1)
b = np.linalg.inv(X_int.T @ X_int) @ X_int.T @ y
print(b)

[ 152.13348416  -10.01219782 -239.81908937  519.83978679  324.39042769
 -792.18416163  476.74583782  101.04457032  177.06417623  751.27932109
   67.62538639]


In [4]:
# Compute estimate of variance sigma**2

preds = X_int @ b
varhat = np.sum((y - preds)**2) / (X_int.shape[0] - X_int.shape[1] - 1)
varb = varhat * np.diag(np.linalg.inv(X_int.T @ X_int))
print(varb)

[6.65044279e+00 3.57826746e+03 3.75688877e+03 4.43700957e+03
 4.28998406e+03 1.74029274e+05 1.15211710e+05 4.52751482e+04
 2.61350159e+04 2.96190102e+04 4.36404173e+03]


In [5]:
# Compute the T-Scores

ts = b / np.sqrt(varb)
print(ts)

[58.99287077 -0.16737594 -3.91263721  7.80412709  4.95267912 -1.89895643
  1.40455451  0.47487908  1.09526528  4.3653208   1.02368258]


In [6]:
# Compute p-values
df = X_int.shape[0] - 1
ps  =[2*(1-stats.t.cdf(np.abs(x), df)) for x in ts]
print(ps)

[0.0, 0.8671509801832249, 0.0001057031114957141, 4.39648317751562e-14, 1.0445511304801869e-06, 0.05822257652567764, 0.16085771224787582, 0.635108244728211, 0.27399823463429795, 1.583097392376942e-05, 0.3065464256087216]


In [7]:
1 - stats.f.cdf(1.67, 4, 58)

0.16927935111708448

In [8]:
# Compute standard errors

(1.96 / 2) * np.sqrt(varb)

array([  2.52726834,  58.62224894,  60.06759504,  65.27866411,
        64.18801049, 408.82479733, 332.63993512, 208.52398498,
       158.43001375, 168.65970875,  64.73967622])

## Gauss-Markov Theorem

Least squares estimates of $\beta$ have the smallest variances among all unbiased estimates.  
Biased estimates (eg Ridge regression) are also usefull

$$\text{MSE}(\hat{\theta}) = \text{Var}(\hat{\theta}) + [\mathbb{E}(\hat{\theta}) - \theta)]^2$$

Variance: $\text{Var}(\hat{\theta})$  
Bias: $[\mathbb{E}(\hat{\theta}) - \theta)]^2$  

Gauss-Makov theorem implies least squares estimator has the smallest MSE among all unbiased estimators.  
But it might exist a biased estimator with a smaller MSE (trade a little bias for larger reduction in variance)

## Regression by orthogonalization

$$\langle x,y \rangle = x^Ty$$


### Univariate regression

$$y = x\beta + \epsilon$$
$$\hat{\beta} = \frac{\langle x,y \rangle}{\langle x,x \rangle}$$

Residual:
$$r = y - x \hat{\beta}$$

$X$ data matrix with columns $x_1$, ..., $x_p$.  
Suppose $X$ colums are orthogonal: $\langle x_j,x_k \rangle = 0 \space \forall j\neq k$
$$\hat{\beta_j} = \frac{\langle x_j,y \rangle}{\langle x_j,x_j \rangle}$$

### Regression by Successive Orthogonalization

For $j = 1, 2, \text{...}, p$:
$$\hat{\gamma_{lj}} = \frac{\langle z_l,x_j \rangle}{\langle z_l,z_l \rangle}$$
$$z_j = x_j - \sum_{k=0}^{j-1} \hat{\gamma_{kj}}z_k$$

$$\hat{\beta_p} = \frac{\langle z_p,y \rangle}{\langle z_p,z_p \rangle}$$

Each $x_j$ is a linear combination of $z_0$, $z_1$, ..., $z_j$.  
All $z_j$ are orthogonal.  
Hence all $z_j$ form a basis for the column space of X

$$\text{Var}(\hat{\beta_p}) = \frac{\sigma^2}{||z_p||^2}$$

Succesive orthogonalization can be represented by:
$$X = Z \Gamma$$  
$Z \in \mathbb{R}^{n*p}$: columns $z_j$  
$\Gamma \in \mathbb{R}^{p*p}$: upper triangular matrix with entries $\gamma_{kj}$

Normalize all $z_j$ by introducting diagonal matrix $D$:
$$D_{jj} = ||z_j||$$

$$X = Z D^{-1} D \Gamma$$
$$X = QR$$

This is the $QR$ decomposition.  
Find orthogonal basis for column space of $X$.  

$$\hat{\beta} = R^{-1}Q^Ty$$
$$\hat{y} = QQ^ty$$

### QR Decomposition by Gram-Schmidt

In [10]:
import sys
sys.path.append('..')
import metrics

In [11]:
def qr(X):
    n = X.shape[0]
    p = X.shape[1]
    
    Q = np.zeros((n, p))
    R = np.eye(p)
    
    for j in range(p):
        for l in range(j):
            R[l, j] = (Q[:,l] @ X[:,j]) / (Q[:,l] @ Q[:,l])
        Q[:,j] = X[:,j]
        for k in range(j):
            Q[:, j] -= R[k, j] * Q[:, k]
        
    D = np.zeros(p)
    for j in range(p):
        D[j] = np.linalg.norm(Q[:,j])
    
    Q = Q @ np.diag(1/D)
    R = np.diag(D) @ R
    return Q, R


X = np.random.randn(13, 5)
y = np.random.randn(13)
Q, R = qr(X)
bp = Q[:, -1] @ y #no need to divide by <z_j,z_j> (already normalized)
print(metrics.tdist(Q @ R, X))
print(metrics.tdist(Q.T @ Q, np.eye(X.shape[1])))

b = np.linalg.inv(R) @ Q.T @ y


print(b[-1])
print(bp)

1.081449488321716e-15
5.72324159413454e-16
0.005083550671442089
0.013463898998340329


## Multiple outputs

Outputs $Y_1$, $Y_2$, $Y_K$.  
One linear model for each output.

$$Y_k = \sum_{j=0}^p X_j \beta_{jk} + \epsilon_k$$
$$Y = XB + E$$

$$RSS(B) = \sum_{k=1}^K \sum_{i=1}^N (y_{ik} - f_k(x_i))^2$$
$$\hat{B} = (X^TX)^{-1})X^TY$$

## Subset Selection

Only retain a subset of the variables, and eliminate the rest from the model. Coefficients of retained inputs are computer with least squares.

### Best-Subset Selection

Find for $k \in {0, 1, 2, \text{...}, p}$ the subset that gives the smallest RSS.  
Test all possible subsets, feasible up to $p=30$.  
Chose smallest model that mninimizes an estimate of prediction error.  
We can use for example cross-validator.

### Forward stepwise selection

Start with the intercept, and at each each adds to the model the predictor that most improves the fit.  
Greedy, suboptimal algorithm.  
Computational cost much lower than best-subset, and less variance because the search is more constrained.

### Backward stepwise selection

Start with full model, and delete predictor that as least impact on fit (eg: smallest z-score).
Contrary to forward, can only be used when  $N > p$

### Forward Stagewise Regression

Start with intercept at $\bar{y}$, and all other coefficients at 0.  
At each step, algorithm identifies the most corelated variable with the current residual, and update it by linear regression.  
It can be very slow to find least squares fit, but can be usefull in high-dimensional problems.

## Shrinkage methods

### Ridge Regression

Shrink coefficient by adding a penalty on their size

$$\hat{\beta} = \arg \min_{\beta} \sum_{i=1}^N (y_i - \sum_{j=0}^p (x_{ij}\beta_j)^2 + \lambda \sum_{j=1}^p \beta_j^2 $$

$\lambda \geq 0$ is the complexity parameter that controls the amount of shrinkage

The problem is equivalent to:

$$\hat{\beta} = \arg \min_{\beta} \sum_{i=1}^N (y_i - \sum_{j=0}^p (x_{ij}\beta_j)^2$$
$$\text{subject to} \sum_{j=1}^p \beta_{j}^2 \leq t$$  

There is a one-to-one correspondance between $t$ and $\lambda$

Input must be normalized before solving.  
$\beta_0$ is not in the penalty term.  

It's possible to center the data first, then apply ridge regression:
Each $x_ij$ is replaced by $x_ij$ - $\bar{x_j}$
$$\beta_0 = \bar{y}$$

$$\text{RSS}(\lambda) = (y - X\beta)^T(y-X\beta) + \lambda \beta^T\beta$$
$$\beta = (X^TX + \lambda I)^{-1} X^Ty$$

In [49]:
X = np.random.randn(13, 5)
y = 4.5 * X[:, 0] + 2.1 * X[:, 1] + 5.7 + 0.1 * X[:, 2]**2
ld = 0.4

Xc = np.mean(X, axis=0, keepdims=True)
yc = np.mean(y)
X2 = X - Xc 
y2 = y - yc

beta = np.linalg.inv(X2.T @ X2 + ld * np.eye(X.shape[1])) @ X2.T @ y2

preds = (X - Xc) @ beta + yc 
err = np.mean((y - preds) ** 2)
print(beta)
print(yc)
print(err)

[ 4.22630102  1.82940796  0.08406004  0.11733299 -0.21012907]
7.113856877098789
0.06999764974598609


Makes the problem non singular, even $X^X X$ is not full rank.

### Least Squares with SVD

$$X = U D V^T$$
$U \in \mathbb{R}^{N*p}$, spans column space of $X$.  
$V \in \mathbb{R}^{p*p}$, spans row space of $X$.  
$D \in \mathbb{R}^{p*p}$, diagonal matrix with singular values of $X$

$$X \beta = X(X^TX)^{-1}X^Ty$$
$$X \beta = UU^Ty$$
$$V^T \beta = z \text{, with } z_i = \frac{u_i^T y}{\sigma_i}$$

$\beta$ found by solving system

In [60]:
X = np.random.randn(117, 3)
y = 4.5*X[:, 0] + 2.1*X[:, 1] + 5.7 + np.random.randn(117) * 0.01

Xc = np.mean(X, axis=0, keepdims=True)
yc = np.mean(y)
X2 = X - Xc 
y2 = y - yc

u, s, vt = np.linalg.svd(X2, full_matrices=False)
z = u.T @ y2 / s
beta = np.linalg.solve(vt, z)

preds = (X - Xc) @ beta + yc 
err = np.mean((y - preds) ** 2)
print(beta)
print(yc)
print(err)

[4.50079410e+00 2.10046285e+00 4.78984296e-04]
6.619639848300166
0.00010847269231719683


$$X \beta = X(X^TX + \lambda I)^{-1}X^Ty$$
$$X \beta = \sum_{j=1}^p u_j \frac{\sigma_j^2}{\sigma_j^2 + \lambda}u_j^Ty$$

$$ \frac{\sigma_j^2}{\sigma_j^2 + \lambda} \leq 1$$
Ridge regression compute the coordinates of $y$ with respect to the orthogonal basis $U$, and shrik it by $\frac{\sigma_j^2}{\sigma_j^2 + \lambda}$.

Sample covariance matrix $S = \frac{X^T}{N}$

$$X^TX = VD^2V^T$$

This is the eigein decomposition of $X^TX$. The eigeinvectors $v_j$ are the principal component directions of $X$

$$z_1 = X v_1 = u_1d_1$$

is the first principal component of $X$. It has the largest sample variance among all the column space of $X$

$$\text{Var}(z_i) = \text{Var}(X v_1) = \frac{\sigma_1^2}{N}$$

Small singular values correspond to directions in column space of $X$ having small variance, and ridge regression shrink these directions the most.

#### Effective degrees of freedom

$$\text{df}(\lambda) = \text{tr} [X(X^TX + \lambda I)^{-1}X^T]$$
$$\text{df}(\lambda) = \text{tr} (H_\lambda)$$
$$\text{df}(\lambda) = \sum_{j=1}^p \frac{\sigma_j^2}{\sigma_j^2 + \lambda}$$  

$$\lambda = 0 \implies \text{df}(\lambda) = p$$
$$\lim_{\lambda \to \infty} \text{df}(\lambda) = 0$$

### Lasso

$$\hat{\beta} = \arg \min_{\beta} \frac{1}{2} \sum_{i=1}^N (y_i - \sum_{j=1}^p (x_{ij}\beta_j)^2 + \lambda \sum_{j=1}^p |\beta_j| $$

The problem is equivalent to:

$$\hat{\beta} = \arg \min_{\beta} \sum_{i=1}^N (y_i - \sum_{j=1}^p (x_{ij}\beta_j)^2$$
$$\text{subject to} \sum_{j=1}^p |\beta_{j}| \leq t$$

The constraint makes the solution non linear, and there is no closed-form solution.  
It's a quadratic programming problem.

Making $t$ small enogh cause some of the coefficients to be $0$.  
Thus lasso does a kind of continious subset selection.

If $t > t_0 = \sum_{j=1}^p |\hat{\beta_j}|$, with $\hat{\beta_j}$ coefficients of linear regression, then lasso gives the same coefficient.
For $t = t_0/2$, the least squares coefficient are shrunk by 50\% on average.

#### Generalization of Ridge and Lasso

$$\hat{\beta} = \arg \min_{\beta} \sum_{i=1}^N (y_i - \sum_{j=1}^p (x_{ij}\beta_j)^2 + \lambda \sum_{j=1}^p |\beta_j|^q $$

for $q \geq 0$.

- $q = 0$: variable selection
- $q = 1$: lasso
- $q = 2$: ridge regression

Corresponds to Bayes estimates with different priors.

#### Elastic Penalty

$$\lambda \sum_{j=1}^p (\alpha \beta_j^2 + (1 - \alpha) |\beta_j|)$$

Compromise between ridge and lasso

### Least Angle Regression

In [102]:
def lar(X, y):
    n = X.shape[0]
    p = X.shape[1]
    
    used = list()
    unused = list(range(p))
    
    beta = np.zeros((p,))
    
    def find_gamma(X_a, beta_a, r, d, j):
        if not len(unused):
            return 1.
        
        step = 0.001
        gamma = 0

        while True:
            corr_j = np.abs((r - X_a @ (beta_a + gamma * d)) @ X[:, j])
            corr_k = np.abs((r - X_a @ (beta_a + gamma * d)) @ X[:, unused])
            if corr_j < np.max(corr_k):
                break
            gamma += step
        
        return gamma
    
    for i in range(p):
        
        #compute residual
        r = y - X @ beta
        
        #find x^j, unused variable most correlated with residual 
        #add j to parameters
        corr = r @ X
        j = unused[np.argmax(np.abs(corr[unused]))]
        used.append(j)
        unused.remove(j)
        
        X_a = X[:, used]
        beta_a = beta[used]
        d = np.linalg.inv(X_a.T @ X_a) @ X_a.T @ r
        
        gamma = find_gamma(X_a, beta_a, r, d, j)
        
        beta[used] += gamma * d
    
    return beta


X = np.random.randn(13, 3) * 2.56 + 3.6
y = 1.34 * X[:, 0] + 2.67 * X[:, 1] + 7.89 + 0.01 * np.random.randn(len(X))
Xc = np.mean(X, axis=0, keepdims=True)
Xs = np.std(X, axis=0, keepdims=True)
yc = np.mean(y)
ys = np.std(y)
X2 = (X - Xc) / Xs
y2 = (y - yc) / ys
beta = lar(X2, y2)

preds = (((X - Xc) / Xs) @ beta) * ys + yc
print('coeffs', beta)
print('error:', np.mean((y - preds)**2))

coeffs [ 6.23402126e-01  7.15671519e-01 -2.47114517e-04]
error: 2.4735593116935953e-05
